# Projeto de Telefonia Móvel
### Este projeto tem por objetivo extrair os dados da fatura detalhada da operadora e transformar os dados em informações importantes e insights sobre a conta

##### Importando as bibliotecas necessárias

In [3]:
!pip install pdfplumber
!pip install pandas
!pip install numpy
!pip install openpyxl
!pip install pyodbc
#pip install PyPDF2
#pip install pyodbc
!pip install PyPDF2
import pdfplumber
import re
import pandas as pd
from pathlib import Path
import numpy as np
import pyodbc

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import os
from PyPDF2 import PdfReader, PdfWriter as PdfFileWriter

def split_pdf(in_pdf, step=50):
    """Splits a given pdf into seperate pdfs and saves 
    those to a supfolder of the parent pdf's folder, called
    splitted_pdf.

    Arguments:
        in_pdf: [str] Absolute path (and filename) of the
                input pdf or just the filename, if the file
                is in the current directory.
        step:   [int] Desired number of pages in each of the
                output pdfs.
    Returns:
        dunno yet
    """
    #TODO: Add choice for output dir
    #TODO: Add logging instead of prints
    #TODO: Refactor
    try:    
        with open(in_pdf, 'rb') as in_file:
            input_pdf = PdfReader(in_file)
            num_pages = len(input_pdf.pages)
            input_dir, filename = os.path.split(in_pdf)
            filename = os.path.splitext(filename)[0]
            output_dir = input_dir + "/" + filename + "_splitted/"
            os.mkdir(output_dir)
            intervals = range(0, num_pages, step)
            intervals = dict(enumerate(intervals, 1))
            naming = f'{filename}_p'
            count = 0
            for key, val in intervals.items():
                output_pdf = PdfFileWriter()
                if key == len(intervals):
                    for i in range(val, num_pages):
                        output_pdf.add_page(input_pdf.pages[i])
                    nums = f'{val + 1}' if step == 1 else f'{val + 1}-{val + step}'
                    with open(f'{output_dir}{naming}{nums}.pdf', 'wb') as outfile:
                        output_pdf.write(outfile)
                    print(f'{naming}{nums}.pdf written to {output_dir}')
                    count += 1
                else:
                    for i in range(val, intervals[key + 1]):
                        output_pdf.add_page(input_pdf.pages[i])
                    nums = f'{val + 1}' if step == 1 else f'{val + 1}-{val + step}'
                    with open(f'{output_dir}{naming}{nums}.pdf', 'wb') as outfile:
                        output_pdf.write(outfile)
                    print(f'{naming}{nums}.pdf written to {output_dir}')
                    count += 1
    except FileNotFoundError as err:
        print('Cannot find the specified file. Check your input:')
    print(f'{count} pdf files written to {output_dir}')
pdf_url='/home/paula/Downloads/VIVO.pdf'    
split_pdf(pdf_url)

# entrada
pasta = '/home/paula/Downloads/VIVO_splitted/'  # '.' = diretorio atual, '..' => anterior
extensoes = ['pdf'] # deixe em branco para todos



VIVO_p1-50.pdf written to /home/paula/Downloads/VIVO_splitted/
1 pdf files written to /home/paula/Downloads/VIVO_splitted/


In [6]:
# lê arquivos na pasta
arquivos = os.listdir(pasta)
arquivos.sort()
text=''
# para cada arquivo na pasta
for i in arquivos:
    print('Entrei')
# se vazio, exibir todos
    pdf_url='/home/paula/Downloads/VIVO_splitted/'f'{i}'
    print(pdf_url)
    with pdfplumber.open(pdf_url) as pdf:
        pages = pdf.pages        
        for i,pg in enumerate(pages):
            text+= pages[i].extract_text()
with Path('/home/paula/Downloads/exemplo.txt').open(mode='w') as output_file:
    output_file.write(text)  

Entrei
/home/paula/Downloads/VIVO_splitted/VIVO_p1-50.pdf


In [7]:
inicio_fatura='DETALHAMENTO TOTAL DA CONTA'
fim_fatura='VEJA O USO DETALHADO DO VIVO 13-99678-3056 - PRUMO ENGENHARIA LTDA'
#fim_fatura='DESCONTOS E PROMOÇÕES'
busca= r'^(\d{2}-\d{5}-\d{4})\s+(.*?)\s+(\d+,\d{2})'
buscaNet=r'^\d{2}-\d{5}-\d{4} \d+MB \d+KB$'
#padrao = 

#correspondencia = re.match(padrao, texto)
dadosFatura=''
dadosGerais=''
texto=''
parcelamento=''
dadosConsumo=''
    #Para sair do laço encontro o final fatura                
with open('/home/paula/Downloads/exemplo.txt', 'r', encoding = 'utf-8') as arquivo:
    
        for linha in arquivo:
            if fim_fatura in linha:
                break
            else:
                    if not("Utilizados") in linha:
                        if(re.search(r'^(\d{2}-\d{5}-\d{4})\s+(.*?)\s+(\d+,\d{2})',linha))and not 'PARCELAMENTO' in linha:
                            if not (re.search(r'\s+\d+KB',linha)):
                                texto=linha
                                strug=' TE'
                                strug1='MAS'
                                strug2='BOX PJ'
                                strug3='BASE'
                                if strug3 in linha: 
                                        linha= linha.replace(' PJ ', ' ')
                                if strug2 in linha: 
                                        linha= linha.replace(' PJ ', ' ')
                                if strug in linha:
                                        linha= linha.replace(' TE ', ' ')
                                if strug1 in linha:
                                        linha= linha.replace(' MAS ', ' ')
                                dadosFatura+=linha

                        if(re.search('^\d{2}-\d{5}-\d{4} \d+MB \d+KB \d{2}-\d{5}-\d{4} \d+MB \d+KB$',linha)):
                            #print(linha)
                            dadosConsumo+=linha
                        if(re.search(r'\s+\d+KB',linha)):
                            dadosConsumo+=linha
                        if(re.search('^Parcelamento de Aparelho / Acessório',linha)):
                            #print(linha)
                            parcelamento+=linha
                    
                  
        #print(dadosFatura)
        with Path('/home/paula/Downloads/dadosGeraisPlanos.txt').open(mode='w') as output_file:
            output_file.write(dadosGerais)       
        with Path('/home/paula/Downloads/dadosFatura.txt').open(mode='w') as output_file:
            output_file.write(dadosFatura)
        with Path('/home/paula/Downloads/dadosConsumos.txt').open(mode='w') as output_file:
            output_file.write(dadosConsumo)
        with Path('/home/paula/Downloads/parcelamentos.txt').open(mode='w') as output_file:
            output_file.write(parcelamento)
            

#### Como a conta detalhada possui muitas páginas, para este projeto foi necessário salvar um arquivo apenas com os dados de cada conta

#### Os dados foram importados do pdf com a biblioteca pdflumber, depois transformados em txt com o auxílio do Path


#### Depois de extraídos, os dados precisavam ser padronizados. Com o auxílio da biblioteca RE, foi criado uma expressão regular para localizar os textos que continham formatos diferentes, logo depois foram removidas informações irrelevantes do plano para este estudo através do método replace.

#### Os dados depois de tratados foram armazenados em arquivos diferentes para fazer a composição de dois dataframes bases relacionados à fatura.

In [8]:

# Ler o arquivo de texto
with open('/home/paula/Downloads/dadosFatura.txt', 'r') as file:
    lines = file.readlines()

# Inicializar listas para os dados
numeros = []
planos = []
valores = []

# Processar as linhas do arquivo
for line in lines:
    # Dividir a linha em palavras
    words = line.split()
    
    # Verificar se a linha possui dados suficientes
    if len(words) >= 10:
        numero = words[0]
        plano = ' '.join(words[1:4])
        valor = words[4]
        numero1 = words[5]
        plano1 = ' '.join(words[6:9])
        valor1 = words[9]
        numeros.append(numero)
        planos.append(plano)
        valores.append(valor)
        
        
        numeros.append(numero1)
        planos.append(plano1)
        valores.append(valor1)

# Criar um DataFrame com os dados processados
dfFatura = pd.DataFrame({'Número': numeros, 'Plano': planos, 'Valor': valores})

# Exibir o DataFrame resultante
print(dfFatura)

            Número                 Plano   Valor
0    13-99678-3056    PLANO INTERNET BOX  186,76
1    24-98124-6525  SMART EMPRESAS 0.2GB   35,32
2    24-98125-2580  SMART EMPRESAS 0.2GB   35,32
3    24-98140-6666  SMART EMPRESAS 0.2GB   35,32
4    24-98140-6667  SMART EMPRESAS 0.2GB   35,32
..             ...                   ...     ...
793  94-99126-9996    SMART EMPRESAS 8GB   70,64
794  94-99128-3722    SMART EMPRESAS 2GB   36,44
795  94-99158-7826    SMART EMPRESAS 8GB   70,64
796  94-99160-0452    SMART EMPRESAS 2GB   36,44
797  94-99239-5650    SMART EMPRESAS 2GB   36,44

[798 rows x 3 columns]


In [9]:
#dfFatura=dfFatura.drop_duplicates()

dfFatura.loc[dfFatura['Número']=='37-99955-6810']

,Número,Plano,Valor
641,37-99955-6810,PLANO BASE INTERNET,"84,99"


In [10]:
dfPlan = pd.read_csv('/home/paula/Downloads/dadosGeraisPlanos.txt', sep="\r\n", header=None, names=["dados"]) #names=["number", "tipo", "c"]
divisao=dfPlan["dados"].str.split(" ") 
desc=divisao.str.get(0)
desc0=divisao.str.get(1)
internet=divisao.str.get(2)
quantidade=divisao.str.get(3)
dfPlan['desc1']=desc
dfPlan['desc2']=desc0
dfPlan['Descrição'] = dfPlan['desc1'] + " " + dfPlan['desc2']
dfPlan['Internet']=internet
dfPlan['Quantidade']=quantidade
dfPlan['Quantidade']=dfPlan['Quantidade'].astype('int32')
dfPlan.drop(['dados'],axis=1,inplace=True)
dfPlan.drop(['desc1'],axis=1,inplace=True)
dfPlan.drop(['desc2'],axis=1,inplace=True)
dfPlan.head()


/tmp/ipykernel_4378/1764421724.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfPlan = pd.read_csv('/home/paula/Downloads/dadosGeraisPlanos.txt', sep="\r\n", header=None, names=["dados"]) #names=["number", "tipo", "c"]


,Descrição,Internet,Quantidade


In [11]:
soma = dfPlan['Quantidade'].sum()
print(soma)

0


#### Agora, iremos trabalhar com o consumo de internet móvel, retirando de outra parte do pdf as informações

In [12]:
dfNet1 = pd.read_csv('/home/paula/Downloads/dadosConsumos.txt', sep="\r\n", header=None, names=["coluna"])

dfNet2=dfNet1.copy()             
divisao=dfNet1["coluna"].str.split(" ")  
dfNet2.head(20)

/tmp/ipykernel_4378/2045110599.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfNet1 = pd.read_csv('/home/paula/Downloads/dadosConsumos.txt', sep="\r\n", header=None, names=["coluna"])


,coluna
0,24-98124-6525 7708MB 24KB 24-98125-2580 66MB 3...
1,24-98124-6525 7708MB 24KB 24-98125-2580 66MB 3...
2,24-98140-6666 138MB 396KB 24-98140-6667 429MB ...
3,24-98140-6666 138MB 396KB 24-98140-6667 429MB ...
4,24-98140-6668 75MB 783KB 24-98140-6669 1619MB ...
5,24-98140-6668 75MB 783KB 24-98140-6669 1619MB ...
6,24-98140-6766 32843MB 772KB 24-98140-6776 3542...
7,24-98140-6766 32843MB 772KB 24-98140-6776 3542...
8,24-98140-6800 917MB 877KB 24-98140-6886 548MB ...
9,24-98140-6800 917MB 877KB 24-98140-6886 548MB ...


In [13]:
    numero_tel=divisao.str.get(0)
    #print(divisao.str.get(2))
    #numero_tel2=divisao.str.get(3)
    qtd_MB=divisao.str.get(1)
    qtd_KB=divisao.str.get(2)
    #qtd_MB2=divisao.str.get(4)
    #qtd_KB2=divisao.str.get(5)
    dfNet1['Numero']=numero_tel
    dfNet1['ConsumoMB']= qtd_MB.str.rstrip('MB').astype('float64')  #remove a unidade de medida e transforma em float
    dfNet1['ConsumoKB']=qtd_KB.str.rstrip('KB').astype('float64')
    #dfNet2['Numero']=numero_tel2
    #dfNet2['ConsumoMB']=qtd_MB2.str.rstrip('MB').astype('float64')
    #dfNet2['ConsumoKB']=qtd_KB2.str.rstrip('KB').astype('float64')
    #dfNet= pd.concat([dfNet1, dfNet2]).sort_values(by='Numero')
    dfNet1.describe()
    #dfNet.drop(['coluna'],axis=1,inplace=True)
    dfNet1['ConsumoGB']= round((dfNet1['ConsumoMB']+(dfNet1['ConsumoKB']/1000))/1000,2)
    dfNet1.head()

,coluna,Numero,ConsumoMB,ConsumoKB,ConsumoGB
0,24-98124-6525 7708MB 24KB 24-98125-2580 66MB 3...,24-98124-6525,7708.0,24.0,7.71
1,24-98124-6525 7708MB 24KB 24-98125-2580 66MB 3...,24-98124-6525,7708.0,24.0,7.71
2,24-98140-6666 138MB 396KB 24-98140-6667 429MB ...,24-98140-6666,138.0,396.0,0.14
3,24-98140-6666 138MB 396KB 24-98140-6667 429MB ...,24-98140-6666,138.0,396.0,0.14
4,24-98140-6668 75MB 783KB 24-98140-6669 1619MB ...,24-98140-6668,75.0,783.0,0.08


In [14]:
dfFatura.loc[dfFatura['Número']=='31-99982-7828']

,Número,Plano,Valor
20,31-99982-7828,SMART EMPRESAS 25GB,"218,60"


#### Nesta etapa, vamos trabalhar os parcelamentos da fatura

In [15]:
dfParc = pd.read_csv('/home/paula/Downloads/parcelamentos.txt', sep="\r\n", header=None, names=["coluna"])
divisao=dfParc["coluna"].str.split(" ") 
numero_tel=divisao.str.get(5)
parcela=divisao.str.get(6)
valor= divisao.str.get(7)
valor=valor.str.replace(',','.')
dfParc['Numero']=numero_tel
dfParc['Parcela']=parcela
dfParc['Valor']=valor.astype('float64')
dfParc.drop(['coluna'],axis=1,inplace=True)
dfParc.head()
#total=dfParc['Valor'].sum()
#print(total)

/tmp/ipykernel_4378/2750821476.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfParc = pd.read_csv('/home/paula/Downloads/parcelamentos.txt', sep="\r\n", header=None, names=["coluna"])


,Numero,Parcela,Valor
0,37-99984-1280,19/24,25.0
1,37-99987-8850,19/24,25.0
2,37-99955-6810,19/24,25.0
3,13-99678-3056,18/24,25.0


In [16]:
dfFatura.describe()
#soma = dfFatura['Valor'].sum()+100
#print(soma)

#dfParc.head()

,Número,Plano,Valor
count,798,798,798
unique,798,10,38
top,13-99678-3056,SMART EMPRESAS 3GB,"35,32"
freq,1,325,218


In [17]:
with pd.ExcelWriter(r'/home/paula/Documents/VIVO.xlsx') as writer:  
    dfFatura.to_excel(writer, sheet_name='Fatura')
    dfParc.to_excel(writer, sheet_name='Parcelamentos')
    dfNet1.to_excel(writer,sheet_name='Consumo Net')
    dfPlan.to_excel(writer,sheet_name='Planos')

#### Ainda resta trazer os valores indevidos. Ainda a pensar sobre